In [ ]:
# Importar las librerías necesarias para manipulación de datos y construcción de modelos
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from preprocess import preprocess_image

# Cargar los datos desde un archivo CSV
data = pd.read_csv('../data/data.csv')

# Preprocesar todas las imágenes y preparar las etiquetas
images = np.array([preprocess_image(img_path) for img_path in data['image_path']])
labels = data['label'].apply(lambda x: 1 if x == 'melanoma' else 0).values

# Dividir los datos en conjuntos de entrenamiento y validación
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Definir el modelo de Deep Learning
def create_model():
    input_shape = (150, 150, 1)  # Imagen en escala de grises
    
    inputs = layers.Input(shape=input_shape)
    
    # Primera capa convolucional
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Segunda capa convolucional
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # Tercera capa convolucional
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Flatten()(x)
    
    # Capa de salida
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=inputs, outputs=output)
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Crear el modelo
model = create_model()

# Entrenar el modelo
history = model.fit(train_images, train_labels, epochs=30, validation_data=(val_images, val_labels))

# Guardar el modelo entrenado
model.save('../models/melanoma_model.h5')
